In [11]:
import pandas as pd
import numpy as np
import pickle

In [12]:
fixtures = pd.read_pickle("../../data/processed/fixtures_with_ratings_latest.pkl")

In [18]:
first_10 = fixtures.loc[fixtures.gw < 11].rename(columns={'elo_outcome_rating_1':'elo_outcome_rating_p',
                                                         'elo_outcome_rating_2':'elo_outcome_rating_o',
                                                         'elo_outcome_rating_diff':'elo_outcome_rating_d',
                                                         'elo_outcome_rating_e':'elo_outcome_rating_ex'})

In [14]:
players_latest = pd.read_pickle('../../data/raw/cleaned_players_latest.pkl')

In [15]:
gameweek_df = pd.read_pickle('../../data/gameweek_with_ma.pkl')

In [16]:
model_cv = pickle.load(open('../../lightgbm_all_data.pkl', 'rb'))
MODEL_VALS = pickle.load(open('../../lightgbm_model_cols.pkl','rb'))

In [17]:
first_10.columns

Index(['elo_outcome_rating_1', 'elo_outcome_rating_2', 'gw', 'is_home',
       'kickoff_time', 'opp_team', 'team', 'elo_outcome_rating_diff',
       'elo_outcome_rating_e'],
      dtype='object')

In [48]:
prev_season_df = (gameweek_df.sort_values(by='Date').drop_duplicates(['clean_name'],keep='last')[['Date','clean_name','team','pos']+MODEL_VALS]
.drop(columns=['gw','value','elo_outcome_rating_p',
 'elo_outcome_rating_o',
 'elo_outcome_rating_d',
 'elo_outcome_rating_ex',]))

In [49]:
players_latest['clean_name'] = players_latest.first_name + '_' + players_latest.second_name

In [50]:
combined_df = prev_season_df.merge(players_latest.loc[:,['selected_by_percent','now_cost','clean_name']],on='clean_name')

In [51]:
forecast_df = first_10.merge(combined_df,on='team')

In [52]:
forecast_df['value'] = forecast_df.now_cost

In [53]:
forecast_df['transfers_in'] = (forecast_df.transfers_in.mean() / forecast_df.selected_by_percent.mean() ) * forecast_df.selected_by_percent
forecast_df['transfers_balance'] = forecast_df.transfers_in - forecast_df.transfers_out

In [54]:
forecast_df['score_forecast'] = model_cv.predict(forecast_df[MODEL_VALS])

In [55]:
selection_df = forecast_df.groupby(['clean_name','pos'])[['score_forecast','value','selected_by_percent']].mean().reset_index().sort_values(by='score_forecast',ascending=False)

In [56]:
selection_df.to_pickle('../../data/processed/live_selections.pkl')

In [60]:
selection_df

,clean_name,pos,score_forecast,value,selected_by_percent
355,Trent_Alexander-Arnold,DEF,6.978821,70,28.7
324,Sadio_Mané,MID,6.891938,115,24.8
320,Ryan_Fraser,MID,6.483821,75,21.4
222,Lucas_Digne,DEF,6.323681,60,27.8
167,Jamie_Vardy,FWD,5.867495,90,11.3
2,Aaron_Wan-Bissaka,DEF,5.767731,55,32.4
66,Chris_Wood,FWD,5.720009,65,1.2
378,Youri_Tielemans,MID,5.591600,65,5.7
305,Ricardo Domingos_Barbosa Pereira,DEF,5.518839,60,8.0
22,Andrew_Robertson,DEF,5.470639,70,36.2
